In [1]:
!nvidia-smi

Sun Sep 19 06:05:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/tensorflow/models.git
%cd models/research/
# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install --use-feature=2020-resolver .
!python object_detection/builders/model_builder_tf2_test.py

Cloning into 'models'...
remote: Enumerating objects: 62926, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 62926 (delta 65), reused 182 (delta 63), pack-reused 62738
Receiving objects: 100% (62926/62926), 574.58 MiB | 36.23 MiB/s, done.
Resolving deltas: 100% (43877/43877), done.
/content/models/research
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 8.5 MB/s 
     |████████████████████████████████| 352 kB 72.0 MB/s 
     |████████████████████████████████| 1.8 MB 62.9 MB/s 
     |██████████

In [4]:
# Jumlah traning steps, 1000 akan training dengan sangat cepat, tetapi lebih banyak steps akan meningkatkan akurasi.
num_steps = 10000  # 200000 steps jika ingin meningkatkan lebih baik

# Jumlah evaluation steps.
num_eval_steps = 50 

MODELS_CONFIG = {
    'ssd_mobilenet_v2_fpn': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_640x640_coco17',
        'pipeline_file': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config',
        'batch_size': 16,
        'checkpoint_path' : '/content/drive/MyDrive/colabData/SSD_MobileNet/checkpoint/ckpt-0'
    },
    'faster_rcnn_resnet152_v1': {
        'model_name': 'faster_rcnn_resnet152_v1_640x640_coco17',
        'pipeline_file': 'faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.config',
        'batch_size': 4,
        'checkpoint_path' : '/content/drive/MyDrive/colabData/F-RCNN/checkpoint/ckpt-0'
    },
    'ssd_resnet152_v1_fpn': {
        'model_name': 'ssd_resnet152_v1_fpn_640x640_coco17',
        'pipeline_file': 'ssd_resnet152_v1_fpn_640x640_coco17_tpu-8.config',
        'batch_size': 16,
        'checkpoint_path' : '/content/drive/MyDrive/colabData/SSD_ResNet/checkpoint/ckpt-0'
    }
}

# Pilih model yang ingin yang dipakai
# Pilih sebuah model dengan ambil dari > `MODELS_CONFIG`.
selected_model = 'ssd_resnet152_v1_fpn'
print("Selected Model: {}".format(selected_model))

# Nama model deteksi objek yang akan digunakan.
MODEL = MODELS_CONFIG[selected_model]['model_name']
print("Model: {}".format(MODEL))

# Nama file pipeline di API deteksi objek tensorflow.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']
print("Pipeline Config File: {}".format(pipeline_file))

# Ukuran Training Batch yang cocok dengan memori GPU Tesla K80 milik Colab untuk model yang dipilih.
batch_size = MODELS_CONFIG[selected_model]['batch_size']
print("Batch Size: {}".format(batch_size))

# Checkpoint pre-trained model
fine_tune_checkpoint = MODELS_CONFIG[selected_model]['checkpoint_path']
print("Pretrained Model Path: {}".format(fine_tune_checkpoint))

Selected Model: ssd_resnet152_v1_fpn
Model: ssd_resnet152_v1_fpn_640x640_coco17
Pipeline Config File: ssd_resnet152_v1_fpn_640x640_coco17_tpu-8.config
Batch Size: 16
Pretrained Model Path: /content/drive/MyDrive/colabData/SSD_ResNet/checkpoint/ckpt-0


In [5]:
# PENTING!: Connect ke drive dahulu sebelum mulai cell ini
# PENTING!: Update nama-nama TFRecord berikut dari "cells" and "cells_label_map" ke file yang dipunya!

train_record_fname = '/content/drive/MyDrive/colabData/train.record'
test_record_fname = '/content/drive/MyDrive/colabData/test.record'
label_map_pbtxt_fname = '/content/drive/MyDrive/colabData/label_map.pbtxt'

In [6]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/configs/tf2', pipeline_file)
 
assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [7]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [8]:
import re
 
num_classes = get_num_classes(label_map_pbtxt_fname)

if num_classes > 1:
  learning_type = "classification"
else:
  learning_type = "detection"

with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # file tfrecord train dan test.
    s = re.sub(
        '(input_path: ".*?)(train2017)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val2017)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)
 
    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)
 
    # Atur training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)
 
    # Atur training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Atur Jumlah classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    # Atur Checkpoint Type
    s = re.sub('fine_tune_checkpoint_type: ".*?"', 'fine_tune_checkpoint_type: "{}"'.format(learning_type), s)
    
    f.write(s)

In [9]:
# Edit Main Model Eval Time
python_file = "/content/models/research/object_detection/model_main_tf2.py"
with open(python_file) as f:
    s = f.read()
with open(python_file, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('wait_interval=300',
               'wait_interval=600', s)
    
    f.write(s)

In [10]:
!cat {python_file}

# Lint as: python3
# Copyright 2020 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

r"""Creates and runs TF2 object detection models.

For local training/evaluation run:
PIPELINE_CONFIG_PATH=path/to/pipeline.config
MODEL_DIR=/tmp/model_outputs
NUM_TRAIN_STEPS=10000
SAMPLE_1_OF_N_EVAL_EXAMPLES=1
python model_main_tf2.py -- \
  --model_dir=$MODEL_DIR --num_train_steps=$NUM_T

In [11]:
!cat {pipeline_fname}

# SSD with Resnet 152 v1 FPN feature extractor, shared box predictor and focal
# loss (a.k.a Retinanet).
# See Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 35.6 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 1
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7


In [12]:
%cd /content/drive/MyDrive/colabData/

/content/drive/MyDrive/colabData


In [13]:
model_dir = 'trained_model/{}/'.format(selected_model)
!rm -r /content/drive/MyDrive/colabData/trained_model/{selected_model}/eval

In [15]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --checkpoint_dir={model_dir} \
    --alsologtostderr

2021-09-19 06:16:19.878140: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-19 06:16:19.887057: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-19 06:16:19.887718: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
W0919 06:16:19.891664 139976889247616 model_lib_v2.py:1082] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0919 06:16:19.891886 139976889247616 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat